In [9]:
%load_ext autoreload
%autoreload 2
 # then imported files are u pdated each time a cell is executed (good if functions in py files are added and modified)

import scipy.io
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import pickle
import sys
import os

os.add_dll_directory("C:/OpenSim4.4/bin") # otherwise module _sombody not found error!

import opensim as osim
from scipy.spatial.transform import Rotation as R

sys.path.insert(0, '..')
sys.path.insert(0, '...')

from Python.plotUtil import addArrow, addMarker
from Python.LMC_util import plotLMC, make_LMC,  check_LMC_Hand_visibility
from Python.Hand_model import getHandModel, getHandMarkers, setHandPose, plot_hand, generate_hand_pose, generate_hand_poses
from Python.ray_tracing_util import get_regressionPlane

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Plot the OSIM Hand

In [ ]:
HandModel = getHandModel()
q = np.zeros(20)
# set the thumb in a flat position
q[0] = 0.4
q[1] = -0.6
q[2] = 0.1
state = setHandPose(HandModel, q)
# HAND ROTATION
# x = + flexion, - extension
# y = + pronation, - supination
# z = + ulna devation, - radial deviation
pos_markers = getHandMarkers(HandModel, state, pos=np.array([0, 0, 0]), orient=np.array([0, 0, 0])*np.pi/180) 

print(pos_markers[24] - pos_markers[25])

fig = go.Figure()
plot_hand(None, pos_markers).show()

[5.20087260e+01 1.87555610e+00 1.86423717e-04]


# Make and Plot some LMC
the grey arrow points in the LMC forward direction

In [7]:
fig = go.Figure()

LMC_H = 600
LMC_alpha1 = (180-150) / 2 * np.pi / 180
LMC_alpha2 = (180-120) / 2 * np.pi / 180

LMC0 = make_LMC(LMC_loc=[0,0,0], LMC_orient=[0, 0, -45], LMC_H=LMC_H, LMC_alpha1=LMC_alpha1, LMC_alpha2=LMC_alpha2)
LMC1 = make_LMC(LMC_loc=[5000,0,0], LMC_orient=[-20, 0, 45], LMC_H=LMC_H, LMC_alpha1=LMC_alpha1, LMC_alpha2=LMC_alpha2)
LMC2 = make_LMC(LMC_loc=[5000,5000,0], LMC_orient=[20, 0, -45], LMC_H=LMC_H, LMC_alpha1=LMC_alpha1, LMC_alpha2=LMC_alpha2)
LMC3 = make_LMC(LMC_loc=[0,5000,0], LMC_orient=[20, 0, 45], LMC_H=LMC_H, LMC_alpha1=LMC_alpha1, LMC_alpha2=LMC_alpha2)

fig = plotLMC(fig, LMC0, color='red', name='0')
fig = plotLMC(fig, LMC1, color='green', name='1')
fig = plotLMC(fig, LMC2, color='blue', name='2')
fig = plotLMC(fig, LMC3, color='orange', name='3')

fig.show()

# Play with raytracing

In [17]:
# GENERAL
HandModel = getHandModel()
q = np.zeros(20)
# set the thumb in a flat position
q[0] = 0.4
q[1] = -0.6
q[2] = 0.1
state = setHandPose(HandModel, q)
fingers_idx = np.array([[0,3,8,17], #thumb (root to tip)
                        [4,9,13,18], #index
                        [5,10,14,19], #middle
                        [6,11,15,20], #ring
                        [7,12,16,21]]) #pinky
finger_radius = 10 # mm
#palm_plane_idx = np.array([7, 6, 5, 4, 1, 25, 24, 2]) # cirsumverence of palm clockwise

# HAND
# x = + flexion, - extension
# y = + pronation, - supination
# z = + ulna devation, - radial deviation
pos_markers = getHandMarkers(HandModel, state, pos=np.array([0, -100, 300]), orient=np.array([0., 90, 0.]))

# PALM;
pv__ = (pos_markers[2] + pos_markers[1]) / 2 - pos_markers[6] # vector pointing from the finger root to the wrist
pv__ /= np.linalg.norm(pv__)
palm_markers = np.array([pos_markers[7] + pv__*3, pos_markers[6] + pv__*3, pos_markers[5] + pv__*3, pos_markers[4] + pv__*3, # to make sure the finger root markers do not intersect with the palm
                         pos_markers[1], pos_markers[25], pos_markers[24], pos_markers[2]])
# normal plane to the palm and palm centroid
palm_plane_normal = get_regressionPlane(palm_markers)
palm_centroid = np.mean(palm_markers, axis=0)

# FINGERS
# the line chain of a finger starts at the finger root and clinbs marker by marker to the finger tip
# the base of a line is always the origin of a vector pointing from towards the tip -> this means the only frogner marker not a line origin is the finger tip
#finger_lines has the shape (num_fingers, num_phlanges, len_phlanges)
finger_lines = np.array([[(pos_markers[fingers_idx[j, i]], (pos_markers[fingers_idx[j, i+1]]-pos_markers[fingers_idx[j, i]])/np.linalg.norm(pos_markers[fingers_idx[j, i+1]]-pos_markers[fingers_idx[j, i]]), np.linalg.norm(pos_markers[fingers_idx[j, i+1]]-pos_markers[fingers_idx[j, i]])) for i in range(fingers_idx.shape[1]-1)] for j in range(fingers_idx.shape[0])])
forearm_vec = (pos_markers[24] + pos_markers[25])/2 - (pos_markers[22] + pos_markers[23])/2

# LMC
LMC = make_LMC(LMC_loc=[0, 0, 0], LMC_orient=[0, 0, 0], LMC_H=600)

fig = go.Figure()
fig = plotLMC(fig, LMC, color='orange', name='LMC', scale=1)
fig = plot_hand(fig, pos_markers)

occlusions, _ = check_LMC_Hand_visibility(LMC, pos_markers, fingers_idx, finger_lines, finger_radius, palm_plane_normal, palm_centroid, palm_markers, forearm_vec, fig=fig, verbose=1)
print(occlusions)
fig.show()

___________________________________
Finger  0
___________________________________
Finger  1
___________________________________
Finger  2
    Finger 2, marker 10: occluded by Finger 1
    Finger 2, marker 14: occluded by Finger 1
___________________________________
Finger  3
    Finger 3, marker 11: occluded by Finger 2
    Finger 3, marker 15: occluded by Finger 2
    Finger 3, marker 20: occluded by Finger 2
___________________________________
Finger  4
    Finger 4, marker 12: occluded by Finger 3
    Finger 4, marker 16: occluded by Finger 3
    Finger 4, marker 21: occluded by Finger 3
[1. 1. 1. 1. 0. 0. 0.]


C:\Users\ge37del\AppData\Local\Temp\ipykernel_8484\360348711.py:36: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.

d:\Dokumente\Projects\LMC_Optim\Python\LMC_util.py:238: RuntimeWarning:

invalid value encountered in arccos



In [39]:
handMarkers, palmMarkers, palmPlnNormals, palmCentroids, fingers = generate_hand_pose(pos=np.array([0, -100, 300]), PS=90, FE=0, WD=0)
data = [handMarkers, palmMarkers, palmPlnNormals, palmCentroids, fingers]
LMCs = [make_LMC(LMC_loc=[-380,  130, 0], LMC_orient=[0, 0, 0], LMC_H=LMC_H) ]
i = 0
print(check_LMC_Hand_visibility(LMCs[0], pos_markers=data[0][i], fingers_idx=fingers_idx, finger_lines=data[4][0], finger_radius=finger_radius, palm_plane_normal=data[2][i], palm_centroid=data[3][i], palm_markers=data[1][i]))

fig = go.Figure()
fig = plotLMC(fig, LMCs[0], color='orange', name='LMC', scale = 30)
fig = plot_hand(fig, data[0][0])
fig.show()

[1. 1. 1. 1. 1. 1. 1.]
